## 欢迎进入 ModelWhale Notebook  

这里你可以编写代码，文档  

### 关于文件目录  


**project**：project 目录是本项目的工作空间，可以把将项目运行有关的所有文件放在这里，目录中文件的增、删、改操作都会被保留  


**input**：input 目录是数据集的挂载位置，所有挂载进项目的数据集都在这里，未挂载数据集时 input 目录被隐藏  


**temp**：temp 目录是临时磁盘空间，训练或分析过程中产生的不必要文件可以存放在这里，目录中的文件不会保存  


In [273]:
# 查看个人持久化工作区文件
!ls /home/mw/project/

In [274]:
# 查看当前挂载的数据集目录
!ls /home/mw/input/

quant4533


In [275]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sklearn 
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import scipy
from scipy import stats
from scipy.stats import pearsonr
from scipy.stats import zscore
from sklearn.linear_model import LinearRegression

In [276]:
plt.rcParams['figure.figsize']=[10,5]

# Get the Data

## Download the Data

In [277]:
df=pd.read_csv('/home/mw/input/quant4533/ruc_Class25Q1_train.csv')
df.head(1)

,城市,区域,板块,环线,小区名称,价格,房屋户型,所在楼层,建筑面积,套内面积,房屋朝向,建筑结构,装修情况,梯户比例,配备电梯,别墅类型,交易时间,交易权属,上次交易,房屋用途,房屋年限,产权所属,抵押信息,房屋优势,核心卖点,户型介绍,周边配套,交通出行,lon,lat,年份
0,0,79.0,111.0,二至三环,人定湖西里,6564200,2室1厅1厨1卫,中楼层 (共5层),52.3㎡,NaN,南 北,混合结构,精装,一梯三户,无,NaN,2020-08-11,商品房,2013-07-31,普通住宅,满五年,非共有,NaN,装修、房本满五年,此房是南北通透小板楼，户型方正，格局合理,房子是南北通透户型方正采光好，前后没有遮挡视野好，通风效果好,医院、公园、超市，生活便利，火箭军医院、积水潭医院，双秀公园，人定湖公园，物美超市、世纪华联等。,NaN,116.389326,39.963727,2018.0


## brief look at the data structure

In [278]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84133 entries, 0 to 84132
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   城市      84133 non-null  int64  
 1   区域      84133 non-null  float64
 2   板块      84133 non-null  float64
 3   环线      42726 non-null  object 
 4   小区名称    84133 non-null  object 
 5   价格      84133 non-null  int64  
 6   房屋户型    83528 non-null  object 
 7   所在楼层    84133 non-null  object 
 8   建筑面积    84133 non-null  object 
 9   套内面积    25146 non-null  object 
 10  房屋朝向    84133 non-null  object 
 11  建筑结构    83528 non-null  object 
 12  装修情况    83528 non-null  object 
 13  梯户比例    82438 non-null  object 
 14  配备电梯    75818 non-null  object 
 15  别墅类型    749 non-null    object 
 16  交易时间    84133 non-null  object 
 17  交易权属    84133 non-null  object 
 18  上次交易    55180 non-null  object 
 19  房屋用途    84131 non-null  object 
 20  房屋年限    54351 non-null  object 
 21  产权所属    84133 non-null  object 
 22

In [279]:
df.hist(bins=50)

array([[<AxesSubplot:title={'center':'城市'}>,
        <AxesSubplot:title={'center':'区域'}>,
        <AxesSubplot:title={'center':'板块'}>],
       [<AxesSubplot:title={'center':'价格'}>,
        <AxesSubplot:title={'center':'抵押信息'}>,
        <AxesSubplot:title={'center':'lon'}>],
       [<AxesSubplot:title={'center':'lat'}>,
        <AxesSubplot:title={'center':'年份'}>, <AxesSubplot:>]],
      dtype=object)

<Figure size 720x360 with 9 Axes>

In [280]:
# 每次随机查看几行数据，逐渐熟悉数据内容
# 设置显示选项确保所有列和行完整显示
pd.set_option('display.max_columns', None)  # 显示所有列
pd.set_option('display.max_colwidth', None)  # 不截断列内容
pd.set_option('display.width',None)
pd.set_option('display.expand_frame_repr', False)  # 不换行显示

df.sample(5)[['price/m2','建筑面积','装修情况','环线','房屋优势','核心卖点','户型介绍','交通出行']]

KeyError: "['price/m2'] not in index"

## create a test set  
+ stratified sampling based on the city category  


In [187]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder

In [188]:
# 将城市特征转化为分层抽样可以接收的离散的数值标签
city_encoder = LabelEncoder()
city_labels = city_encoder.fit_transform(df['城市'])

In [189]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=111)

for train_index, test_index in split.split(df, city_labels):
    df_train = df.iloc[train_index]
    df_test = df.iloc[test_index]

# Discover and Visualize the Data to Find Correlations

In [190]:
demo=df_train.copy()

## numeric attributes  
+ 建筑面积：包含公摊面积   公摊比例高的房子（如高层电梯房）  
+ 套内面积：是购房者实际使用的空间，通常与房价呈强正相关  
建筑面积 = 套内面积+公摊面积  
+ 模型解释性：套内面积对房价的解释力更强，而建筑面积可能因包含噪音（公摊差异）导致解释性下降  
+ 共线性：两者高度相关，必要时降维或保留其一  


#### 套内面积or建筑面积： 哪个更适合成为预测特征

In [191]:
# 将带单位的套内面积和建筑面积转化为数值类型
demo['套内面积']=demo['套内面积'].str.extract('(\d+\.?\d*)').astype(float)

In [192]:
demo['得房率']=demo['套内面积']/demo['建筑面积']
demo['得房率'].describe()
# 可知得房率的波动在7%以内，得房率较为稳定；数据中房屋套内面积与建筑面积的差别不大

In [193]:
# 抽取套内面积非空的行，查看套内面积和建筑面积哪个对房价的解释力更高
df_non_missing=demo.dropna(subset=['套内面积']).copy()
# 计算相关系数
corr_construction=df_non_missing['建筑面积'].corr(df_non_missing['价格'],method='spearman')
corr_interior=df_non_missing['套内面积'].corr(df_non_missing['价格'],method="spearman")
corr_construction, corr_interior


In [194]:
# 散点图可视化
sns.pairplot(df_non_missing,vars=['建筑面积','套内面积','价格'],diag_kind='kde')

In [195]:
# 可以看出建筑面积与房价的相关系数与套内面积相差极小（甚至略强于套内面积；可能因为中国市场的计价习惯以及人们考虑公摊面积的价值贡献）
# 因此为避免共线性问题，删除“套内面积”只保留“建筑面积”作为预测特征
demo.drop(columns=['套内面积'],inplace=True)    # 表示直接修改原数据

#### 处理“建筑面积”特征

In [196]:
sns.scatterplot(x='建筑面积',y='价格',data=demo,alpha=0.6)

In [197]:
lower_thresh=demo['建筑面积'].quantile(0.0001)
upper_thresh=demo['建筑面积'].quantile(0.9999)
outlier_percentile=demo[(demo['建筑面积']<lower_thresh)|(demo['建筑面积']>upper_thresh)]
outlier_percentile
# 貌似存在错误数据： 面积过大 房价过低

In [198]:
from sklearn.preprocessing import FunctionTransformer

def remove_outliers(X, y, threshold=27):    # 从threshold=3 开始，观察剔除的点是否合理
    model = LinearRegression().fit(X, y)
    residuals = y - model.predict(X)
    z_scores = np.abs(zscore(residuals))
    is_outlier = z_scores>threshold
    return (is_outlier)
is_outlier=remove_outliers(demo[['建筑面积']],demo['价格'])  
demo[is_outlier]
# 最终在threshold=27时，remove_outlier可以成功筛选出在散点图中观察到的两个明显离群的错误数据

In [199]:
outlier_remover=FunctionTransformer(
    lambda df: remove_outliers('建筑面积','价格',threshold=27)
)

#### numeric pipeline

In [200]:
def convert_m2(area_series):
    return area_series.str.replace('㎡', '').astype(float)

m2_transformer = FunctionTransformer(
    lambda df: pd.DataFrame(convert_m2(df['建筑面积']), columns=['建筑面积'])
)

In [201]:
numeric_pipe = Pipeline([
    ('m2_convert', m2_transformer),
    ('outlier_removal', outlier_remover)
])

#### 之后可能的尝试方向：1.预测每平米单价*建筑面积   or 2.建筑面积作为主要特征直接预测总价

In [202]:
# 房屋面积是否差异过大？
demo['建筑面积'].describe()

In [203]:
demo['price/m2'].describe()

##  categorial attributes  
**类型变量大致有六类**  
+ 是否类型：配备电梯；别墅类型；产权所属  
+ 种类少顺序特征：房屋年限；装修情况；环线  
+ 种类少名义特征：房屋用途；建筑结构   
+ 种类多名义特征：小区名称；房屋户型  
+ 种类所顺序特征：所在楼层  
+ 地理信息：城市；区域；板块; 小区名称  

**由于时间关系，只研究了2个种类少的顺序变量并总结了处理种类少的顺序变量的pipeline**

### 类别少有顺序

**房屋年限**  
+ “满五年”的房子比“满两年”的房子房价通常更高3%-5%，因买方节省的个税成本会部分转化为溢价  
+ 房价基数越高，税费差异对总价的影响更大 →应在相似价格水平的情况下再考虑房屋年限的影响？）  
+ missing: 29782   推断：未填的说明房屋年限方面没有优势  


In [204]:
demo['房屋年限'].value_counts(dropna=False)

In [205]:
demo['房屋年限']=demo['房屋年限'].fillna('空白')

In [206]:
median_price1=demo.groupby('房屋年限')['price/m2'].mean().reset_index()
median_price1

In [207]:
plt.figure(figsize=(10, 6))
sns.boxplot(
    data=demo,
    x='房屋年限',
    y='price/m2',
    )
plt.xticks()

In [208]:
# 可以看出‘满五年‘的房屋的均价明显高于其他类别，满两年和未满两年之间并无明显差别
#’空白‘房屋的均价似乎更低与满两年与未满两年，但差距并不明显，谨慎起见没有将’空白‘单独编码
order_map = {
    '满五年': 1,  
    '满两年': 0,
    '未满两年': 0,
    '空白': 0  
}

demo['holding_year_01'] = demo['房屋年限'].map(order_map)

**装修情况**  
+ 最终处理方案：1. 将特征缺失值填补为‘空白’  2.按照设定{‘精装’=1，‘简装’=1，‘毛坯’=0，‘其他’=0，‘空白’=0}对特征进行01编码

In [209]:
demo['装修情况'].value_counts()

In [210]:
demo['装修情况']=demo['装修情况'].fillna('空白')

In [211]:
median_price2=demo.groupby('装修情况')['price/m2'].mean().reset_index()
median_price2

In [212]:
plt.figure(figsize=(10, 6))
sns.boxplot(
    data=demo,
    x='装修情况',  
    y='price/m2',  
    order=['毛坯', '简装', '精装', '其他','空白']
)

plt.xticks()

In [213]:
# 观察到’简装‘和’精装‘类别每平米房价普遍较高，而’毛坯‘’其他‘’空白‘类别每平米房价集中在低价，因此考虑将’装修情况‘进行01编码，精装简装设定为1，毛坯其他和空白设定为0来体现这种阈值效应
order_map = {'毛坯':0, '其他':0, '空白':0, '简装':1, '精装':1}
demo['decoration_01'] = demo['装修情况'].map(order_map)
demo['decoration_01'].value_counts()

#### 类别较少的顺序变量的pipeline

In [214]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

# 1. 定义每个特征的编码函数
def encode_装修情况(series):
    mapping = {'精装': 1, '简装': 1, '毛坯': 0, '其他': 0, '空白': 0}
    return series.fillna('未填').map(mapping)

def encode_房屋年限(series):
    mapping = {'满五年': 1, '未满两年': 0, '满两年': 0, '空白': 0}
    return series.fillna('未填').map(mapping)

# 2. 创建转换器
cat_transformers = [
    ('装修情况', FunctionTransformer(lambda df: encode_装修情况(df['装修情况']).to_frame(), validate=False), ['装修情况']),
    ('房屋年限', FunctionTransformer(lambda df: encode_房屋年限(df['房屋年限']).to_frame(), validate=False), ['房屋年限'])
]

In [215]:
cat_transformers

#### 地理信息  
+ 城市  
+ 区域  
+ 板块  
+ lon  
+ lat  
+ 小区名称  


In [216]:
demo1=df_train.copy()

In [217]:
demo['城市']=demo['城市'].astype('category')
demo['区域']=demo['区域'].astype('category')
demo['板块']=demo['板块'].astype('category')

In [218]:
# 查看各个城市是否有相同的区域
# 获取每个城市的区域集合
城市区域 = demo.groupby('城市')['区域'].apply(set)

# 检查所有区域集合的交集
所有区域 = set().union(*城市区域.values)
区域计数器 = demo['区域'].value_counts()

重复区域 = 区域计数器[区域计数器 > 1].index.tolist()
重复区域

#### 城市  


In [219]:
city_data=demo.groupby('城市').agg({'lon':['min','max'],'lat':['min','max'],'价格':'mean'}).reset_index()
city_data.columns=['城市','min_lon','max_lon','min_lat','max_lat','平均房价']
city_data

In [220]:
city_data.plot(kind='bar',x='城市',y='平均房价')

In [221]:
city_data.plot(kind='scatter',x='min_lon',y='min_lat',c='平均房价',s=100,cmap='jet',colorbar=True,legend=True,grid=True)

In [222]:
demo_city=demo[['城市']]
demo_city.head(2)

In [223]:
city_encoder=OneHotEncoder()
demo_city_1hot=city_encoder.fit_transform(demo_city)

In [224]:
city_pipe = Pipeline([
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

#### 区域   
+ 仅进行了初步的数据探索，未形成最终的预测特征  


In [225]:
fig,(ax1,ax2)=plt.subplots(2,1,figsize=(12,32))
pivot1=demo.pivot_table(values='价格',index='区域',columns='城市',aggfunc='mean')
sns.heatmap(pivot1,ax=ax1,annot=True,cmap='jet')
# pivot2=demo.pivot_table(values='价格',index='板块',columns='区域',aggfunc='mean')
# sns.heatmap(pivot2,ax=ax2,annot=True,cmap='jet')

In [226]:
# 创建 2x3 的子图画布
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
cities = demo["城市"].unique()

for ax, city in zip(axes.flatten(), cities):
    # 筛选当前城市数据
    city_data = demo[demo["城市"] == city]
    # 生成区域-房价矩阵（热力图数据）
    pivot_table = city_data.pivot_table(index="区域", values="价格", aggfunc=np.mean)
    sns.heatmap(pivot_table, annot=True, fmt=".1f", cmap="jet", ax=ax)
    ax.set_title(f"{city}城市各区域房价（万元/㎡）")

plt.tight_layout()

##### 板块  
+ 仅进行了初步的数据探索 未形成最终的预测特征

In [227]:
city_median_price=demo1.groupby(['城市','区域'])['价格'].median().reset_index()
city_median_price['价格'].describe()

In [228]:
district_median_price=demo1.groupby(['城市'])['价格'].median().reset_index()
district_median_price['价格'].describe()

In [229]:
block_median_price=demo1.groupby(['城市','区域','板块'])['价格'].median().reset_index()
block_median_price['价格'].describe()

In [230]:
block_median_price.head()

In [231]:
# 生成透视矩阵
pivot_data = block_median_price.pivot_table(
    index=["城市", "区域"],
    columns="板块",
    values="价格"
)

# 绘制热力图
plt.figure(figsize=(12, 6))
sns.heatmap(pivot_data, annot=False, fmt=".1f", cmap="jet")
plt.title("各城市-区域-板块房价热力图（万元/㎡）")

In [232]:
# 将小区名称替换为该小区的平均房价
# 逻辑链条：如果小区名称对房价影响大→不同小区的均值差异会很大→生产的‘小区均价’与‘实际房价’高度相关

## text attributes  
+ 房屋优势：missing 16064  unique27 多变为统一，描述了小区的装修/房屋年限/地铁状况  
剩下的text attributes特征均为：缺失值较多，内容多变  
+ 核心买点：missing 16366  
+ 户型介绍 missing 63671  
+  周边配套 missing 34027  
+  交通出行 missing 32437  

最终处理方案： 由于01编码处理下text attribute 和房价的相关性并不明显，最终未将其作为预测变量

In [233]:
layout_intro=demo['户型介绍'].value_counts()
layout_intro.head(10)
# 可以看出“户型介绍”大部分是正向积极评价 并非客观中立评价

#### 变量编码  
+ 可以看出所有自然语言描述都是对房屋的正向积极评价，且除"房屋优势"外，其余变量内容较为复杂多样，不容易提取关键词分类。因此考虑将各个变量缺失值设定为”0“，非缺失值设定为”1“  
**不足及改进方向**  
+ 忽略了文本内容差异—— 关键词标记编码  对关键短语（如”地铁“、”学区“）进行二进制标记；TF-IDF+将为（复杂文本）  
+ 无法量化强度——文本长度编码 描述越详细，可能反映信息越重要；文本情感强度编码  正向描述的强烈程度可能影响房价  
+ 难以捕捉交互效应：多个text特征同时存在可能对房价有叠加影响

In [234]:
# 0/1编码
text_columns=['房屋优势','核心卖点','户型介绍','周边配套','交通出行']
for col in text_columns:
    demo[f'{col}_01']=demo[col].notnull().astype(int)
print(demo[['price/m2']+[f'{col}_01' for col in text_columns]])
print(demo.isnull().sum())    # 确保所有文本列的缺失值已正确处理

In [235]:
# 相关系数矩阵
corr_cols=['price/m2']+[f'{col}_01' for col in text_columns]
corr_matrix = demo[corr_cols].corr(method='spearman')
corr_matrix

In [236]:
# 相关系数矩阵可视化
sns.heatmap(corr_matrix,annot=True,cmap='jet',vmin=-1,vmax=1)

In [237]:
# 相关系数矩阵的结果与我的事先推断并不相符，周边配套和交通出行甚至和房价呈现微弱的负相关
# 通过房价分组检查高价房是否更可能拥有自然语言描述（我的推断）
demo['price_group']=pd.qcut(demo['price/m2'],q=3,labels=['low','median','high'])
print(demo.groupby('price_group')[['交通出行_01','周边配套_01','房屋优势_01','核心卖点_01','户型介绍_01']].mean())
# 最终结果显示高单价房更少填写交通出行以及周边配套，可能因为高价房核心的优势并不在于周边配套和交通出行


# Prepare the Data for Machine Learning

In [238]:
from sklearn.compose import ColumnTransformer
num_attribs=['建筑面积']
cat_attribs=['房屋年限','装修情况']

In [239]:
preprocessor = ColumnTransformer([
    ('numeric', numeric_pipe, num_attribs),      # 数值处理
    ('city', city_pipe, ['城市']),                # 城市独热编码
], remainder='drop')  # 明确丢弃其他列

# Select and Train the model

In [240]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer

In [241]:
from sklearn.model_selection import GridSearchCV

In [242]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [243]:
X_train = df_train.drop('价格', axis=1)
y_train =df_train['价格']
y_mean = y_train.mean()

In [247]:
model_pipeline=Pipeline([
    ('preprocessor',preprocessor),
    ('regressor',LinearRegression())
])

In [248]:
# 训练模型
model_pipeline.fit(X_train, y_train)

# 预测训练集
y_pred = model_pipeline.predict(X_train)

# 计算评估指标
mae = mean_absolute_error(y_train, y_pred)
rmse = np.sqrt(mean_squared_error(y_train, y_pred))

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R² Score: {model_pipeline.score(X_train, y_train):.4f}")

## Model Performance

#### final test

In [13]:
# 读入用于最终测试的csv
df_final_test=pd.read_csv('/home/mw/input/quant4533/ruc_Class25Q1_test.csv')

In [ ]:
# [9] 最终预测
best_model_name = comparison.index[0]
best_model = results[best_model_name]['model']
print(f"\nBest model: {best_model_name}")

final_predictions = best_model.predict(df_final_test)
submission = pd.DataFrame({
    'ID': df_final_test['ID'],
    'predicted_price': final_predictions
})
submission.to_csv('final_predictions.csv', index=False)
print("Predictions saved to final_predictions.csv")